# Import des librairies et conneexion à la base de données

In [1]:
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()
database_url = os.getenv("DATABASE_URL") # url de la base de données stocké dans un fichier .env sous la variable DATABASE_URL
engine = create_engine(database_url) 

ArgumentError: Expected string or URL object, got None

# Récupération de la première table

In [35]:

query = "SELECT tconst, titletype, primarytitle, isadult, startyear, endyear, runtimeminutes, genres, averagerating, numvotes FROM sebastien.title_basics ORDER BY tconst LIMIT 10000;"
with engine.connect() as conn, conn.begin():
    df = pd.read_sql_query(query, engine)

df.head()

,tconst,titletype,primarytitle,isadult,startyear,endyear,runtimeminutes,genres,averagerating,numvotes
0,tt12605172,tvEpisode,Episode #1.2,False,2020.0,NaN,NaN,"Action,Mystery",NaN,NaN
1,tt12605176,tvSpecial,Shaheb Bibi Golam,False,2016.0,NaN,NaN,Drama,NaN,NaN
2,tt12605178,short,Girl in the Mirror,False,2020.0,NaN,6.0,"Music,Short",NaN,NaN
3,tt1260518,tvEpisode,Loose Ends,False,2008.0,NaN,NaN,Drama,NaN,NaN
4,tt12605180,tvEpisode,Episode #1.3,False,2020.0,NaN,NaN,"Action,Mystery",NaN,NaN


On récupère seulement les 10000 premières lignes ordonnées par tconst pour pouvoir par la suite récupérer les données des autres tables selon ces 10000 lignes.

# Récupération de la table episode et join

### Récupération de la table dans la base de données

In [36]:
query = "SELECT * FROM sebastien.title_episode;"
with engine.connect() as conn, conn.begin():
    df_episode = pd.read_sql_query(query, engine)

df_episode.head()

,tconst,parenttconst,seasonnumber,episodenumber
0,tt12155488,tt12145510,1.0,7.0
1,tt1215549,tt0990536,1.0,3.0
2,tt12155490,tt12092480,1.0,211.0
3,tt12155492,tt12092480,1.0,212.0
4,tt12155494,tt12145510,1.0,8.0


La complication est que certaines lignes de title_basics sont liés à title_episode par la colonne tconst de title_episode tandis que d'autres sont liés par la colonne parentTconst.

### Premier merge avec tconst

In [37]:
df_merge = df.merge(df_episode, on="tconst", how="left")
df_merge

,tconst,titletype,primarytitle,isadult,startyear,endyear,runtimeminutes,genres,averagerating,numvotes,parenttconst,seasonnumber,episodenumber
0,tt12605172,tvEpisode,Episode #1.2,False,2020.0,NaN,NaN,"Action,Mystery",NaN,NaN,tt12465816,1.0,2.0
1,tt12605176,tvSpecial,Shaheb Bibi Golam,False,2016.0,NaN,NaN,Drama,NaN,NaN,NaN,NaN,NaN
2,tt12605178,short,Girl in the Mirror,False,2020.0,NaN,6.0,"Music,Short",NaN,NaN,NaN,NaN,NaN
3,tt1260518,tvEpisode,Loose Ends,False,2008.0,NaN,NaN,Drama,NaN,NaN,tt0210405,18.0,5.0
4,tt12605180,tvEpisode,Episode #1.3,False,2020.0,NaN,NaN,"Action,Mystery",NaN,NaN,tt12465816,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,tt12624184,tvEpisode,Canción de cuna,False,2001.0,NaN,NaN,"Crime,Drama,Thriller",NaN,NaN,tt0222600,NaN,NaN
9996,tt12624186,videoGame,Go Diego Go! Great Dinosaur Rescue,False,2008.0,NaN,NaN,Adventure,5.3,7.0,NaN,NaN,NaN
9997,tt1262419,movie,Sanpei the Fisher Boy,False,2009.0,NaN,118.0,Drama,6.1,109.0,NaN,NaN,NaN
9998,tt12624194,movie,Debbie's Time,True,1971.0,NaN,NaN,Adult,NaN,NaN,NaN,NaN,NaN


### Deuxième merge par parenttconst

In [38]:
df_merge_parent = df.merge(df_episode, left_on="tconst", right_on="parenttconst", how="left", suffixes=('', '_parent'))
df_merge_parent

,tconst,titletype,primarytitle,isadult,startyear,endyear,runtimeminutes,genres,averagerating,numvotes,tconst_parent,parenttconst,seasonnumber,episodenumber
0,tt12605172,tvEpisode,Episode #1.2,False,2020.0,NaN,NaN,"Action,Mystery",NaN,NaN,NaN,NaN,NaN,NaN
1,tt12605176,tvSpecial,Shaheb Bibi Golam,False,2016.0,NaN,NaN,Drama,NaN,NaN,NaN,NaN,NaN,NaN
2,tt12605178,short,Girl in the Mirror,False,2020.0,NaN,6.0,"Music,Short",NaN,NaN,NaN,NaN,NaN,NaN
3,tt1260518,tvEpisode,Loose Ends,False,2008.0,NaN,NaN,Drama,NaN,NaN,NaN,NaN,NaN,NaN
4,tt12605180,tvEpisode,Episode #1.3,False,2020.0,NaN,NaN,"Action,Mystery",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21704,tt12624184,tvEpisode,Canción de cuna,False,2001.0,NaN,NaN,"Crime,Drama,Thriller",NaN,NaN,NaN,NaN,NaN,NaN
21705,tt12624186,videoGame,Go Diego Go! Great Dinosaur Rescue,False,2008.0,NaN,NaN,Adventure,5.3,7.0,NaN,NaN,NaN,NaN
21706,tt1262419,movie,Sanpei the Fisher Boy,False,2009.0,NaN,118.0,Drama,6.1,109.0,NaN,NaN,NaN,NaN
21707,tt12624194,movie,Debbie's Time,True,1971.0,NaN,NaN,Adult,NaN,NaN,NaN,NaN,NaN,NaN


### On combine les deux merge

In [39]:
df = pd.concat([df_merge, df_merge_parent], ignore_index=True)
df

,tconst,titletype,primarytitle,isadult,startyear,endyear,runtimeminutes,genres,averagerating,numvotes,parenttconst,seasonnumber,episodenumber,tconst_parent
0,tt12605172,tvEpisode,Episode #1.2,False,2020.0,NaN,NaN,"Action,Mystery",NaN,NaN,tt12465816,1.0,2.0,NaN
1,tt12605176,tvSpecial,Shaheb Bibi Golam,False,2016.0,NaN,NaN,Drama,NaN,NaN,NaN,NaN,NaN,NaN
2,tt12605178,short,Girl in the Mirror,False,2020.0,NaN,6.0,"Music,Short",NaN,NaN,NaN,NaN,NaN,NaN
3,tt1260518,tvEpisode,Loose Ends,False,2008.0,NaN,NaN,Drama,NaN,NaN,tt0210405,18.0,5.0,NaN
4,tt12605180,tvEpisode,Episode #1.3,False,2020.0,NaN,NaN,"Action,Mystery",NaN,NaN,tt12465816,1.0,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31704,tt12624184,tvEpisode,Canción de cuna,False,2001.0,NaN,NaN,"Crime,Drama,Thriller",NaN,NaN,NaN,NaN,NaN,NaN
31705,tt12624186,videoGame,Go Diego Go! Great Dinosaur Rescue,False,2008.0,NaN,NaN,Adventure,5.3,7.0,NaN,NaN,NaN,NaN
31706,tt1262419,movie,Sanpei the Fisher Boy,False,2009.0,NaN,118.0,Drama,6.1,109.0,NaN,NaN,NaN,NaN
31707,tt12624194,movie,Debbie's Time,True,1971.0,NaN,NaN,Adult,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df[df["tconst"].duplicated()] 

,tconst,titletype,primarytitle,isadult,startyear,endyear,runtimeminutes,genres,averagerating,numvotes,parenttconst,seasonnumber,episodenumber,tconst_parent
10000,tt12605172,tvEpisode,Episode #1.2,False,2020.0,NaN,NaN,"Action,Mystery",NaN,NaN,NaN,NaN,NaN,NaN
10001,tt12605176,tvSpecial,Shaheb Bibi Golam,False,2016.0,NaN,NaN,Drama,NaN,NaN,NaN,NaN,NaN,NaN
10002,tt12605178,short,Girl in the Mirror,False,2020.0,NaN,6.0,"Music,Short",NaN,NaN,NaN,NaN,NaN,NaN
10003,tt1260518,tvEpisode,Loose Ends,False,2008.0,NaN,NaN,Drama,NaN,NaN,NaN,NaN,NaN,NaN
10004,tt12605180,tvEpisode,Episode #1.3,False,2020.0,NaN,NaN,"Action,Mystery",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31704,tt12624184,tvEpisode,Canción de cuna,False,2001.0,NaN,NaN,"Crime,Drama,Thriller",NaN,NaN,NaN,NaN,NaN,NaN
31705,tt12624186,videoGame,Go Diego Go! Great Dinosaur Rescue,False,2008.0,NaN,NaN,Adventure,5.3,7.0,NaN,NaN,NaN,NaN
31706,tt1262419,movie,Sanpei the Fisher Boy,False,2009.0,NaN,118.0,Drama,6.1,109.0,NaN,NaN,NaN,NaN
31707,tt12624194,movie,Debbie's Time,True,1971.0,NaN,NaN,Adult,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
df[df["tconst"] == "tt3240414"]

,tconst,titletype,primarytitle,isadult,startyear,endyear,runtimeminutes,genres,averagerating,numvotes,parenttconst,seasonnumber,episodenumber,tconst_parent


On voit qu'il y a des doublons, il faut donc regrouper les lignes doublons

Pour cela on peux regrouper les lignes qui ont le même tconst. Pour les colonnes de title_basics, comme la valeur est la même selon les deux merge, on peux garder toujours le premier. Pour les colonnes de title_episode, pour éliminer les lignes où la valeur est vide, on peux garder la valeur max, qui est donc celle non nulle.

In [42]:
df = df.groupby('tconst').agg({
    'titletype': 'first', # on garde  le titletype de la première ligne
    'primarytitle': 'first',
    'isadult': 'first',
    'startyear': 'first',
    'endyear': 'first',
    'runtimeminutes': 'first',
    'genres': 'first',
    'averagerating': 'first',
    'numvotes': 'first',
    'seasonnumber': 'max', # on garde le max de seasonnumber, soit le non nul
    'episodenumber': 'max'
}).reset_index()
df

,tconst,titletype,primarytitle,isadult,startyear,endyear,runtimeminutes,genres,averagerating,numvotes,seasonnumber,episodenumber
0,tt12605172,tvEpisode,Episode #1.2,False,2020.0,NaN,NaN,"Action,Mystery",NaN,NaN,1.0,2.0
1,tt12605176,tvSpecial,Shaheb Bibi Golam,False,2016.0,NaN,NaN,Drama,NaN,NaN,NaN,NaN
2,tt12605178,short,Girl in the Mirror,False,2020.0,NaN,6.0,"Music,Short",NaN,NaN,NaN,NaN
3,tt1260518,tvEpisode,Loose Ends,False,2008.0,NaN,NaN,Drama,NaN,NaN,18.0,5.0
4,tt12605180,tvEpisode,Episode #1.3,False,2020.0,NaN,NaN,"Action,Mystery",NaN,NaN,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,tt12624184,tvEpisode,Canción de cuna,False,2001.0,NaN,NaN,"Crime,Drama,Thriller",NaN,NaN,NaN,NaN
9996,tt12624186,videoGame,Go Diego Go! Great Dinosaur Rescue,False,2008.0,NaN,NaN,Adventure,5.3,7.0,NaN,NaN
9997,tt1262419,movie,Sanpei the Fisher Boy,False,2009.0,NaN,118.0,Drama,6.1,109.0,NaN,NaN
9998,tt12624194,movie,Debbie's Time,True,1971.0,NaN,NaN,Adult,NaN,NaN,NaN,NaN


# Récupération de la table title_akas et join

On groupe par tconst pour récupérer pour chaque film directement le nombre de région et la liste des régions, pour prendre le moins de place possible en mémoire.

In [8]:
query_title_akas = """
SELECT 
    ta.tconst, 
    COUNT(DISTINCT ta.region) AS regionnumber, 
    ARRAY_AGG(ta.region) AS regionlist
FROM 
    sebastien.title_akas ta
WHERE 
    ta.tconst IN (SELECT tconst FROM sebastien.title_basics ORDER BY tconst LIMIT 10000)
GROUP BY 
    ta.tconst;
"""

with engine.connect() as conn, conn.begin():
    df_akas = pd.read_sql_query(query_title_akas, conn)

df_akas.head()

,tconst,regionnumber,regionlist
0,tt12605172,8,"[PT, IN, FR, IT, DE, JP, \N, ES]"
1,tt12605176,2,"[\N, BD]"
2,tt12605178,2,"[\N, US]"
3,tt1260518,1,[\N]
4,tt12605180,8,"[\N, PT, IN, FR, IT, DE, JP, ES]"


In [11]:
def replace_and_filter(region_list):
    if isinstance(region_list, list):
        return [region for region in region_list if region != '\\N' and region != '']
    return region_list
df_akas['regionlist'] = df_akas['regionlist'].apply(replace_and_filter)
df_akas

,tconst,regionnumber,regionlist
0,tt12605172,8,"[PT, IN, FR, IT, DE, JP, ES]"
1,tt12605176,2,[BD]
2,tt12605178,2,[US]
3,tt1260518,1,[]
4,tt12605180,8,"[PT, IN, FR, IT, DE, JP, ES]"
...,...,...,...
9995,tt12624184,2,[MX]
9996,tt12624186,2,[US]
9997,tt1262419,5,"[XWW, BR, JP, CN]"
9998,tt12624194,2,[US]


In [44]:
df = df.merge(df_akas, on="tconst", how="left")
df

,tconst,titletype,primarytitle,isadult,startyear,endyear,runtimeminutes,genres,averagerating,numvotes,seasonnumber,episodenumber,regionnumber,regionlist
0,tt12605172,tvEpisode,Episode #1.2,False,2020.0,NaN,NaN,"Action,Mystery",NaN,NaN,1.0,2.0,8,"[PT, IN, FR, IT, DE, JP, \N, ES]"
1,tt12605176,tvSpecial,Shaheb Bibi Golam,False,2016.0,NaN,NaN,Drama,NaN,NaN,NaN,NaN,2,"[\N, BD]"
2,tt12605178,short,Girl in the Mirror,False,2020.0,NaN,6.0,"Music,Short",NaN,NaN,NaN,NaN,2,"[\N, US]"
3,tt1260518,tvEpisode,Loose Ends,False,2008.0,NaN,NaN,Drama,NaN,NaN,18.0,5.0,1,[\N]
4,tt12605180,tvEpisode,Episode #1.3,False,2020.0,NaN,NaN,"Action,Mystery",NaN,NaN,1.0,3.0,8,"[\N, PT, IN, FR, IT, DE, JP, ES]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,tt12624184,tvEpisode,Canción de cuna,False,2001.0,NaN,NaN,"Crime,Drama,Thriller",NaN,NaN,NaN,NaN,2,"[\N, MX]"
9996,tt12624186,videoGame,Go Diego Go! Great Dinosaur Rescue,False,2008.0,NaN,NaN,Adventure,5.3,7.0,NaN,NaN,2,"[\N, US]"
9997,tt1262419,movie,Sanpei the Fisher Boy,False,2009.0,NaN,118.0,Drama,6.1,109.0,NaN,NaN,5,"[\N, XWW, BR, JP, CN]"
9998,tt12624194,movie,Debbie's Time,True,1971.0,NaN,NaN,Adult,NaN,NaN,NaN,NaN,2,"[\N, US]"


In [45]:
df["regionlist"] = df["regionlist"].str.replace("\\N", "")
df

,tconst,titletype,primarytitle,isadult,startyear,endyear,runtimeminutes,genres,averagerating,numvotes,seasonnumber,episodenumber,regionnumber,regionlist
0,tt12605172,tvEpisode,Episode #1.2,False,2020.0,NaN,NaN,"Action,Mystery",NaN,NaN,1.0,2.0,8,NaN
1,tt12605176,tvSpecial,Shaheb Bibi Golam,False,2016.0,NaN,NaN,Drama,NaN,NaN,NaN,NaN,2,NaN
2,tt12605178,short,Girl in the Mirror,False,2020.0,NaN,6.0,"Music,Short",NaN,NaN,NaN,NaN,2,NaN
3,tt1260518,tvEpisode,Loose Ends,False,2008.0,NaN,NaN,Drama,NaN,NaN,18.0,5.0,1,NaN
4,tt12605180,tvEpisode,Episode #1.3,False,2020.0,NaN,NaN,"Action,Mystery",NaN,NaN,1.0,3.0,8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,tt12624184,tvEpisode,Canción de cuna,False,2001.0,NaN,NaN,"Crime,Drama,Thriller",NaN,NaN,NaN,NaN,2,NaN
9996,tt12624186,videoGame,Go Diego Go! Great Dinosaur Rescue,False,2008.0,NaN,NaN,Adventure,5.3,7.0,NaN,NaN,2,NaN
9997,tt1262419,movie,Sanpei the Fisher Boy,False,2009.0,NaN,118.0,Drama,6.1,109.0,NaN,NaN,5,NaN
9998,tt12624194,movie,Debbie's Time,True,1971.0,NaN,NaN,Adult,NaN,NaN,NaN,NaN,2,NaN


In [46]:
df.columns

Index(['tconst', 'titletype', 'primarytitle', 'isadult', 'startyear',
       'endyear', 'runtimeminutes', 'genres', 'averagerating', 'numvotes',
       'seasonnumber', 'episodenumber', 'regionnumber', 'regionlist'],
      dtype='object')

# Récupération de la table title_principals et name_basics et join

On récupère les lignes correspondantes aux 10000 premières lignes de title_basics.

In [56]:
from pandas.core.frame import DataFrame


query_title_akas = """
SELECT 
    tconst,
    category,
    primaryname
FROM 
    sebastien.title_principals ta
JOIN 
    sebastien.name_basics nb 
ON 
    ta.nconst = nb.nconst
WHERE 
    ta.tconst IN (SELECT tconst FROM sebastien.title_basics ORDER BY tconst LIMIT 10000);
"""

with engine.connect() as conn, conn.begin():
    df_principals: DataFrame = pd.read_sql_query(query_title_akas, conn)

df_principals.head()

,tconst,category,primaryname
0,tt12605302,actor,Sam Page
1,tt12605302,writer,Celeste Vasquez
2,tt12605304,writer,Celeste Vasquez
3,tt12605530,producer,Jenna Gien
4,tt12605626,actor,Wilson Gonzalez


On groupe les lignes par tconst et category, puis on agrège les primaryname sous forme de liste, et on transforme les category en des colonnes pour chaque category (en remplissant les nulls par des listes vides).

In [57]:

df_principals = df_principals.groupby(['tconst', 'category'])['primaryname'].agg(list).unstack(fill_value=[]).reset_index()

df_principals = df_principals[["tconst", "actor", "self", "producer", "actress", "director"]]

df_final = pd.merge(df, df_principals, on='tconst', how='left')

df_final.head()

,tconst,titletype,primarytitle,isadult,startyear,endyear,runtimeminutes,genres,averagerating,numvotes,seasonnumber,episodenumber,regionnumber,regionlist,actor,self,producer,actress,director
0,tt12605172,tvEpisode,Episode #1.2,False,2020.0,NaN,NaN,"Action,Mystery",NaN,NaN,1.0,2.0,8,NaN,"[Ryoma Takeuchi, Kôtarô Yoshida, Hayato Ichiha...",[],[],[Mikako Tabe],[Eiichirô Hasumi]
1,tt12605176,tvSpecial,Shaheb Bibi Golam,False,2016.0,NaN,NaN,Drama,NaN,NaN,NaN,NaN,2,NaN,"[Omer Ayaz Ony, Fs Nayeem]",[],[Hasan Al Mamun Ponir],[Aparna Ghosh],[Mir Asaduzzaman Ariyan]
2,tt12605178,short,Girl in the Mirror,False,2020.0,NaN,6.0,"Music,Short",NaN,NaN,NaN,NaN,2,NaN,[],[],[Greg L. Hines],[Vernae Taylor],[Greg L. Hines]
3,tt1260518,tvEpisode,Loose Ends,False,2008.0,NaN,NaN,Drama,NaN,NaN,18.0,5.0,1,NaN,"[Paul Akl, Michael Apgar, Alex De Leon, Chris ...",[],[],"[Amanda Garant, Angelica Allen, Leah Garvin]","[Tessa Olson, Jenn Carroll]"
4,tt12605180,tvEpisode,Episode #1.3,False,2020.0,NaN,NaN,"Action,Mystery",NaN,NaN,1.0,3.0,8,NaN,"[Ryoma Takeuchi, Kôtarô Yoshida, Tatsuya Fujiw...",[],[],[Mikako Tabe],[Eiichirô Hasumi]


# Sauvegarde de notre dataset final

In [59]:
df.to_csv("data/all_data_for_10000_lines.csv", index=False)

# Test de la fonction

In [1]:
import pandas as pd
import sys
sys.path.append('..')
from scripts.FeaturesExtraction import get_extracted_features

In [2]:
df = get_extracted_features(10000)
df

table 1 / 4
table 2 / 4
table 3 / 4
table 4 / 4


,tconst,titletype,primarytitle,isadult,startyear,genres,averagerating,numvotes,seasonnumber,episodenumber,regionnumber,regionlist,actor,self,producer,actress,director
0,tt12605172,tvEpisode,Episode #1.2,False,2020.0,"Action,Mystery",NaN,NaN,1.0,2.0,8,"[PT, IN, FR, IT, DE, JP, \N, ES]","[Ryoma Takeuchi, Kôtarô Yoshida, Hayato Ichiha...",[],[],[Mikako Tabe],[Eiichirô Hasumi]
1,tt12605176,tvSpecial,Shaheb Bibi Golam,False,2016.0,Drama,NaN,NaN,NaN,NaN,2,"[\N, BD]","[Omer Ayaz Ony, Fs Nayeem]",[],[Hasan Al Mamun Ponir],[Aparna Ghosh],[Mir Asaduzzaman Ariyan]
2,tt12605178,short,Girl in the Mirror,False,2020.0,"Music,Short",NaN,NaN,NaN,NaN,2,"[\N, US]",[],[],[Greg L. Hines],[Vernae Taylor],[Greg L. Hines]
3,tt1260518,tvEpisode,Loose Ends,False,2008.0,Drama,NaN,NaN,18.0,5.0,1,[\N],"[Paul Akl, Michael Apgar, Alex De Leon, Chris ...",[],[],"[Amanda Garant, Angelica Allen, Leah Garvin]","[Tessa Olson, Jenn Carroll]"
4,tt12605180,tvEpisode,Episode #1.3,False,2020.0,"Action,Mystery",NaN,NaN,1.0,3.0,8,"[\N, PT, IN, FR, IT, DE, JP, ES]","[Ryoma Takeuchi, Kôtarô Yoshida, Tatsuya Fujiw...",[],[],[Mikako Tabe],[Eiichirô Hasumi]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,tt12624184,tvEpisode,Canción de cuna,False,2001.0,"Crime,Drama,Thriller",NaN,NaN,NaN,NaN,2,"[\N, MX]","[Roberto Sen, Sebastián Rulli, Hugo Denis]",[],[],[Tiaré Scanda],"[Felipe Nájera, Francisco Franco Alba]"
9996,tt12624186,videoGame,Go Diego Go! Great Dinosaur Rescue,False,2008.0,Adventure,5.3,7.0,NaN,NaN,2,"[\N, US]","[Keeler Sandhaus, Keeler Sandhaus, Keeler Sand...",[],[],"[Katie McWane, Katie McWane, Michelle Concha, ...",[]
9997,tt1262419,movie,Sanpei the Fisher Boy,False,2009.0,Drama,6.1,109.0,NaN,NaN,5,"[\N, XWW, BR, JP, CN]","[Kenta Suga, Masato Hagiwara, Ryûji Katagiri, ...",[],[Masatake Kondô],"[Yû Kashii, Cynthia Cheston]",[Yôjirô Takita]
9998,tt12624194,movie,Debbie's Time,True,1971.0,Adult,NaN,NaN,NaN,NaN,2,"[\N, US]",[],[],[Sven Peters],[],[Sven Peters]


In [5]:
df.to_csv("../data/all_data_for_10000_lines.csv", index=False)